# 第9回講義 演習

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from tensorflow_graph_in_jupyter import show_graph

rng = np.random.RandomState(1234)

/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 課題1. グラフ上でのLoop

tensorflowの計算グラフ上でloop構造を実現するには, `tf.scan`関数を使用します

#### tf.scan関数
- 主な引数
    - fn: 入力系列に適用する関数
    - elems: 入力系列 (第0軸方向に走査していく)
    - initializer: 最初の引数
    
参考:
https://www.tensorflow.org/api_docs/python/tf/scan

#### tf.scanの機能と注意事項

まず, 入力系列に対して適用する関数fnは, fn(a, x)といった様に, 2つの引数を持つものである必要があります.

この2つの引数にはそれぞれ役割があり, 次のようになっています.
  - 第1引数: 前ステップのfnの出力
  - 第2引数: 今ステップの入力(elems)
  
つまり, 出力される系列は, 例えばelemsの長さがNであれば,

$f_1={\rm fn}(initializer, elems[0])$

$f_2={\rm fn}(f_1, elems[1])$

$f_3={\rm fn}(f_2, elems[2])$

$\vdots$

$f_N={\rm fn}(f_{N-1}, elems[N-1])$

ということになります.

#### 例:Accumulation function for vector

In [2]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32)

def fn(a, x):
    return a + x

res = tf.scan(fn=fn, elems=x)
show_graph(tf.get_default_graph())

In [3]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={x: np.array([1, 2, 3, 4, 5, 6])}))

[ 1.  3.  6. 10. 15. 21.]


#### 例:Accumulation function for matrix

In [4]:
x = tf.placeholder(tf.float32)

def fn(a, x):
    return a + x

res = tf.scan(fn=fn, elems=x)

In [5]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: np.array([[1, 2, 3, 4, 5],
                         [1, 2, 3, 4, 5],
                         [1, 2, 3, 4, 5]])
    }))
    #一番外側のループで中の対応する要素どうしに適応する。

[[ 1.  2.  3.  4.  5.]
 [ 2.  4.  6.  8. 10.]
 [ 3.  6.  9. 12. 15.]]


#### 例: initializer
* tf.scanのinitializerという引数で，loop構造の初期値を明示的に指定します．特にinitializerが指定されない場合は，上記のように入力系列の最初が初期値となります．

In [6]:
x = (tf.placeholder(tf.float32), tf.placeholder(tf.float32))
init = tf.placeholder(tf.float32)

def fn(a, x):
    return x[0] - x[1] + a

res = tf.scan(fn=fn, elems=x, initializer=init)

In [7]:
elems = np.array([1, 2, 3, 4, 5, 6])
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: (elems+1, elems),
            init: np.array(0)
    }))

[1. 2. 3. 4. 5. 6.]


#### 例: フィボナッチ数列（initializerを利用）
$F_0 = 0,$
$F_1 = 1,$
$F_{n + 2} = F_n + F_{n + 1} (n ≧ 0)$

In [8]:
x = tf.placeholder(tf.float32)
init = (tf.placeholder(tf.float32), tf.placeholder(tf.float32))

def fn(a, _):
    return (a[1], a[0]+a[1])

res = tf.scan(fn=fn, elems=x, initializer=init)

# fibonaccis == ([1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13])

In [9]:
with tf.Session() as sess:
    print(sess.run(res, feed_dict={
            x: np.array([0, 0, 0, 0, 0, 0]),
            init: (np.array(0), np.array(1))
    }))

(array([1., 1., 2., 3., 5., 8.], dtype=float32), array([ 1.,  2.,  3.,  5.,  8., 13.], dtype=float32))


## 課題2. Recurrent Neural Network (RNN) によるIMDbのsentiment analysis

IMDb (Internet Movie Database) と呼ばれる映画レビューのデータセットで

各レビュー文の評価がpositiveかnegativeかをRNNを用いて予測してみましょう.

<div style="text-align: center;">【データセットのイメージ】</div>

| レビュー | 評価 |
|:--------:|:-------------:|
|Where's Michael Caine when you need him? I've ...|negative|
|To experience Head you really need to understa...|positive|

※実際には各単語が出現頻度順位で数字に置き換えられたものがXとして, 評価をnegativeなら0, positiveなら1に置き換えたものがyとして入ることになります.

### 1. データセットの読み込み

In [10]:
# 出現頻度上位num_words番までのみを扱う. それ以外は丸ごと1つの定数に置き換え(0:start_char, 1:oov_char, 2~:word_index)
num_words = 10000
(train_X, train_y), (test_X, test_y) = imdb.load_data(num_words=num_words, start_char=0, oov_char=1, index_from=2)

# split data into training and validation
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

# データセットサイズが大きいので演習用に短縮
train_X = train_X[:len(train_X)//2]
train_y = train_y[:len(train_y)//2]
valid_X = valid_X[:len(valid_X)//2]
valid_y = valid_y[:len(valid_y)//2]

In [11]:
train_X #もうすでに単語にユニークなIDが対応済みのデータになっている

array([list([0, 72, 88, 80, 24, 59, 966, 5, 19, 140, 16, 13, 30, 126, 11, 59, 27, 1359, 1106, 65, 44, 5, 19, 14, 496, 7, 78, 16, 490, 7, 111, 5, 6682, 19, 16, 613, 690, 3, 435, 19, 8, 2854, 5, 761, 6, 492, 8620, 5, 184, 249, 23, 54, 2275, 4, 22, 349, 6, 14, 81, 23, 14, 820, 65, 9, 9, 44, 577, 14, 3, 19, 804, 7, 29, 16, 820, 4, 1620, 16, 613, 189, 3, 19, 8, 42, 31, 98, 1213, 17, 14, 7, 156, 45, 16, 1435, 3, 1, 4, 1, 8, 31, 1795, 4, 1213, 266, 16, 72, 16, 4412, 35, 25, 399, 42, 4561, 82, 3, 1872, 246, 73, 82, 3, 249, 539, 81, 3, 95, 3, 249, 8305, 7, 31, 3, 1, 8, 183, 3965, 12, 383, 47, 13, 15, 146, 1347, 58, 61, 68, 9419, 11, 45, 49, 8, 52, 1, 73, 1929, 10, 13, 30, 150, 9, 9, 3, 19, 8, 539, 363, 351, 4, 44, 5, 1, 588, 32, 268, 7, 2714, 141, 1620, 4, 820, 16, 72, 16, 203, 4, 2907, 18, 54, 1762, 4696, 91, 65, 103, 13, 19, 92, 75, 1487, 150, 32, 3, 57, 11, 187, 625, 150, 11, 214, 68, 223, 141, 72, 236, 5, 963, 6, 1445, 2288, 187, 625, 102, 13, 30, 9, 9, 450, 6, 1464, 4, 598, 79, 90, 1328, 2

### 2. 可変長系列のミニバッチ化

IMDbの各データは長さの異なるレビュー (の各単語を出現頻度順で数値化したもの) です. これに対してRNNを適用し, 最後の隠れ層ベクトルを元に二値分類をおこないます.

この問題で異なる長さの系列をミニバッチ化する際には次の2つのことに注意する必要があります.

- ミニバッチ内のデータの系列の長さをpaddingによって揃える.
- paddingした部分の計算を無効にする

#### 2.1. ミニバッチ内のデータの系列の長さをpaddingによって揃える.

異なる系列長のデータをミニバッチ(行列)に落とし込むために, ミニバッチ内の短い系列に対して頭orお尻にpaddingし長さを揃える必要があります. これは `keras` にある関数 `pad_sequences` を使うなどすればできます. またpaddingの量を少なくするために, あらかじめデータの長さで降順にソートしておくことが多いです.

#### 2.2. paddingした部分の計算を無効にする
これは実質的に、可変長への対応といえるのか？

paddingの部分はあくまで系列長を合わせるためなので, 通常のRNNの計算はおこなわず, 何らかの形で計算を無効にする必要があります. ここではわかりやすい実装として, paddingの部分では代わりに前のステップの隠れ層をコピーするようにし, 実際の系列の最後の単語における隠れ層ベクトルを保持するようにします.

具体的には, 各インスタンスに対して実際に単語がある部分に1, ない部分(paddingの部分)に0を置くバイナリのマスク$m=[m_1, m_2, \dots, m_t, \dots, m_T]$をつくり,

$$
    h_t = m_t \cdot \sigma({\bf W_x} x_t + {\bf W_h} h_{t-1} + b) + (1-m_t) \cdot h_{t-1}
$$

とします. こうすることでpaddingの部分では$h_t=h_{t-1}$となり, paddingの計算結果に対する影響がなくなります.

### 3. 各層クラスの実装

#### 3.1. Embedding層

Embedding層では, 単語を離散的なidから連続的な数百次元のベクトルに変換(埋め込み, embed)します.

下のEmbeddingクラスにおいて, 入力`x`は各行に文の単語のid列が入った行列で, 重み`V`は各行がそれぞれの単語idのベクトルに対応した行列です. つまりそれぞれの行列のサイズは

- `x`: (ミニバッチサイズ) x (ミニバッチ内の文の最大系列長)
- `V`: (辞書の単語数) x (単語のベクトルの次元数)

です.

この`V`から, 入力`x`のそれぞれの単語idに対して対応する単語ベクトルを取り出すことで, 各単語をベクトルに変換します. 

`tf`では`tf.nn.embedding_lookup`によりこの作業を行います.この処理によって出力されるテンソルの次元数は，(ミニバッチサイズ) x (ミニバッチ内の文の最大系列長) x (単語のベクトルの次元数)となります（embedding層に関する詳細は，次の第10回講義で説明があります）．

In [12]:
class Embedding:
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        self.V = tf.Variable(rng.randn(vocab_size, emb_dim).astype('float32') * scale, name='V')

    def f_prop(self, x):
        return tf.nn.embedding_lookup(self.V, x)

#### 3.2. RNN

RNNクラスでは, Embedding層で各単語がベクトルに変換されたものを入力として処理を行います. ここで入力`x`は

- `x`: (ミニバッチサイズ) x (ミニバッチ内の文の最大系列長) x (単語のベクトルの次元数)

となっています. `tf.scan`では第0軸方向に走査していくので, 文の系列方向に沿って走査するために上の第0軸と第1軸を入れ替えて

- `x`: (ミニバッチ内の文の最大系列長) x (ミニバッチサイズ) x (単語のベクトルの次元数)

とします.

In [13]:
#  Random orthogonal initializer (see [Saxe et al. 2013])
def orthogonal_initializer(shape, scale = 1.0):
    a = np.random.normal(0.0, 1.0, shape).astype(np.float32)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == shape else v
    return scale * q

In [14]:
class RNN:
    def __init__(self, in_dim, hid_dim, m, scale=0.08):
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        # Xavier initializer
        self.W_in = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(in_dim + hid_dim)),
                        high=np.sqrt(6/(in_dim + hid_dim)),
                        size=(in_dim, hid_dim)
                    ).astype('float32'), name='W_in')
        # Random orthogonal initializer
        self.W_re = tf.Variable(orthogonal_initializer((hid_dim, hid_dim)), name='W_re')
        self.b_re = tf.Variable(tf.zeros([hid_dim], dtype=tf.float32), name='b_re')
        self.m = m

    def f_prop(self, x):
        def fn(h_tm1, x_and_m):
            x = x_and_m[0]
            m = x_and_m[1]
            h_t = tf.nn.tanh(tf.matmul(h_tm1, self.W_re) + tf.matmul(x, self.W_in) + self.b_re)
            return m[:, np.newaxis] * h_t + (1 - m[:, np.newaxis]) * h_tm1 # Mask

        # shape: [batch_size, sentence_length, in_dim] -> shape: [sentence_length, batch_size, in_dim]
        _x = tf.transpose(x, perm=[1, 0, 2])
        # shape: [batch_size, sentence_length] -> shape: [sentence_length, batch_size]
        _m = tf.transpose(self.m)
        h_0 = tf.matmul(x[:, 0, :], tf.zeros([self.in_dim, self.hid_dim])) # Initial state
        
        h = tf.scan(fn=fn, elems=[_x, _m], initializer=h_0)
        
        return h[-1] # Take the last state

In [15]:
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        # Xavier initializer
        self.W = tf.Variable(rng.uniform(
                        low=-np.sqrt(6/(in_dim + out_dim)),
                        high=np.sqrt(6/(in_dim + out_dim)),
                        size=(in_dim, out_dim)
                    ).astype('float32'), name='W')
        self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
        self.function = function

    def f_prop(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

### 4. 計算グラフ構築 & パラメータの更新設定

In [16]:
emb_dim = 100
hid_dim = 50

x = tf.placeholder(tf.int32, [None, None], name='x')
m = tf.cast(tf.not_equal(x, -1), tf.float32) # Mask. Paddingの部分(-1)は0, 他の値は1
t = tf.placeholder(tf.float32, [None, None], name='t')

layers = [
    Embedding(num_words, emb_dim),
    RNN(emb_dim, hid_dim, m=m),
    Dense(hid_dim, 1, tf.nn.sigmoid)
]

def f_props(layers, x):
    for i, layer in enumerate(layers):
        x = layer.f_prop(x)
    return x

y = f_props(layers, x)

cost = tf.reduce_mean(-t*tf.log(tf.clip_by_value(y, 1e-10, 1.0)) - (1. - t)*tf.log(tf.clip_by_value(1.-y, 1e-10, 1.0)))

train = tf.train.AdamOptimizer().minimize(cost)
test = tf.round(y)

### 5. 学習

In [17]:
# Sort train data according to its length
train_X_lens = [len(com) for com in train_X]
sorted_train_indexes = sorted(range(len(train_X_lens)), key=lambda x: -train_X_lens[x])

train_X = [train_X[ind] for ind in sorted_train_indexes]
train_y = [train_y[ind] for ind in sorted_train_indexes]

In [19]:
n_epochs = 5
batch_size = 100
n_batches_train = len(train_X) // batch_size
n_batches_valid = len(valid_X) // batch_size

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for epoch in range(n_epochs):
        # Train
        train_costs = []
        for i in range(n_batches_train):
            start = i * batch_size
            end = start + batch_size
            
            train_X_mb = np.array(pad_sequences(train_X[start:end], padding='post', value=-1)) # Padding
            train_y_mb = np.array(train_y[start:end])[:, np.newaxis]

            _, train_cost = sess.run([train, cost], feed_dict={x: train_X_mb, t: train_y_mb})
            train_costs.append(train_cost)
        
        # Valid
        valid_costs = []
        pred_y = []
        for i in range(n_batches_valid):
            start = i * batch_size
            end = start + batch_size
            
            valid_X_mb = np.array(pad_sequences(valid_X[start:end], padding='post', value=-1)) # Padding
            valid_y_mb = np.array(valid_y[start:end])[:, np.newaxis]
            
            pred, valid_cost = sess.run([test, cost], feed_dict={x: valid_X_mb, t: valid_y_mb})
            pred_y += pred.flatten().tolist()
            valid_costs.append(valid_cost)
        print('EPOCH: %i, Training cost: %.3f, Validation cost: %.3f, Validation F1: %.3f' % (epoch+1, np.mean(train_costs), np.mean(valid_costs), f1_score(valid_y, pred_y, average='macro')))
    
#エラーが出るがどうしてかよくわからん

InvalidArgumentError: indices[96,950] = -1 is not in [0, 10000)
	 [[Node: embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@Adam/Assign_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](V/read, _arg_x_0_1, embedding_lookup/axis)]]

Caused by op 'embedding_lookup', defined at:
  File "/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/